### Table of contents:

* [install required packages](#install)

* [toggle code on off](#toggle)

* [create a dataframe of k8 repos](#repos)
    
* [create a dataframe of k8 issues](#issues)
    
* [merge issues and repos](#merge)

* [create a zenhub connection](#zenhub)

* [extract boards from zenhub](#zencall)

* [get status of all issues from zenhub pipeline](#status)

* [view boards per project](#per-project)

* [view boards per status](#per-status)

* [save boards to csv file](#save)

## import modules <a class="anchor" id="install"></a>

In [1]:
import sys
sys.path.insert(0, '../models/')
from issues import Issues
import config
from transform import transform_issues
from repos import Repos
import pandas as pd
from zenhub import Zenhub
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import interactive
from IPython.display import HTML
import time

## toggle code on off <a class="anchor" id="toggle"></a>

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## extract repos and their id <a class="anchor" id="repos"></a>

In [3]:
repos = Repos('k8-proxy').get_repos()[['name', 'id']]
repos=repos.rename(columns={"id":"repo_id", "name": "repo"})

## extract issues and their id <a class="anchor" id="issues"></a>

In [4]:
issues = transform_issues()[['repo', 'title','number', 'id']]
issues = issues.rename(columns={"id":"issue_id"})

## merge issues to their respective repos <a class="anchor" id="merge"></a>

In [5]:
issue = issues.merge(repos, on='repo')
issue.shape

(252, 5)

## create a Zenhub connection <a class="anchor" id="zenhub"></a>

In [6]:
zh = Zenhub(config.ZENHUB_TOKEN)

## extract issue data from zenhub in batches of 100 <a class="anchor" id="zencall"></a>

In [7]:
issue['pipes'] = ""
for i in range(0, 100):
    issue.pipes[i] = zh.get_issue_data(repo_id=issue.repo_id[i], issue_number=issue.number[i])
time.sleep(60)

In [8]:
for i in range(100, 200):
    issue.pipes[i] = zh.get_issue_data(repo_id=issue.repo_id[i], issue_number=issue.number[i])
time.sleep(60)

In [9]:
for i in range(200, len(issue)):
    issue.pipes[i] = zh.get_issue_data(repo_id=issue.repo_id[i], issue_number=issue.number[i])

## view raw boards

In [10]:
issue.head()

,repo,title,number,issue_id,repo_id,pipes
0,aws-jmeter-test-engine,Implement delete stack feature in main master ...,89,732644826,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
1,aws-jmeter-test-engine,Enable prefix in createStack python script and...,87,731993676,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
2,aws-jmeter-test-engine,Icapmonitoring,84,730095998,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
3,aws-jmeter-test-engine,Create master python script to automatically c...,83,729349759,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
4,aws-jmeter-test-engine,Test data integration,82,729070408,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."


## get the status of issues from zenhub pipelines <a class="anchor" id="status"></a>

In [11]:
issue['status'] = ""
for i in range(0, len(issue)):
    if 'pipelines' in issue['pipes'][i]:
        issue['status'][i] = issue.pipes[i]['pipelines'][0]['name']
    else:
        issue['status'][i] = None
boards = issue.drop('pipes', 1)
boards = boards.rename(columns = {"repo": "project", "title":"summary", "number":"task"})

## boards  <a class="anchor" id="boards"></a>

In [12]:
boards.head(2)

,project,summary,task,issue_id,repo_id,status
0,aws-jmeter-test-engine,Implement delete stack feature in main master ...,89,732644826,298066702,New Issues
1,aws-jmeter-test-engine,Enable prefix in createStack python script and...,87,731993676,298066702,In Progress


## sample for k8-data-visualization

In [13]:
boards.query("project=='k8-data-visualization'").head()

,project,summary,task,issue_id,repo_id,status
135,k8-data-visualization,Extract k8-proxy repo's programmatically,140,731682115,289667349,In Progress
136,k8-data-visualization,Create a new functionality of the github-label...,138,731660893,289667349,Backlog
137,k8-data-visualization,zenhub_boards notebook is showing some error,135,728097585,289667349,Peer Review
138,k8-data-visualization,issues notebook is showing some error,134,728092766,289667349,Peer Review
139,k8-data-visualization,issues notebook is showing some error,122,720423013,289667349,Backlog


## widget to slice the boards per project <a class="anchor" id="per-project"></a>

In [14]:
items = ['All']+sorted(boards['project'].unique().tolist())
def view(Project=''):
    if Project=="All": display(boards)
    display(boards[boards.project==Project].reset_index(drop=True))
w = widgets.Select(options=items)

In [15]:
interactive(view, Project=w)

interactive(children=(Select(description='Project', options=('All', 'GW-proxy', 'ICAP-threat-model', 'aws-jmet…

## widget to slice the boards per status <a class="anchor" id="per-status"></a>

In [16]:
stats = boards['status'].unique().tolist()
stat = [x for x in stats if x is not None]
items = ['All']+sorted(stat)
def view(Status=''):
    if Status=="All": display(boards)
    display(boards[boards.status==Status].reset_index(drop=True))
w = widgets.Select(options=items)

In [17]:
interactive(view, Status=w)

interactive(children=(Select(description='Status', options=('All', 'Backlog', 'Done', 'In Progress', 'New Issu…

## save boards to file <a class="anchor" id="save"></a>

In [18]:
# save to csv
boards.to_csv("../data/boards.csv", index=False)